<a href="https://colab.research.google.com/github/NAJOO0/2022F-Ajou-ML/blob/main/2023AI_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install torch torchvision

In [2]:
#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
args={}
kwargs={}
args['batch_size']=128
args['test_batch_size']=128
args['epochs']=5  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=5000 // args['batch_size']
args['cuda']=False

In [4]:
print(args)

{'batch_size': 128, 'test_batch_size': 128, 'epochs': 5, 'lr': 0.01, 'momentum': 0.5, 'seed': 1, 'log_interval': 39, 'cuda': False}


In [5]:
# #load the data
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Resize((224,224)),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args['batch_size'], shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Resize((224,224)),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args['test_batch_size'], shuffle=True, **kwargs)


In [6]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(root='MNIST_data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('MNIST_data/', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

100%|██████████| 26421880/26421880 [00:01<00:00, 15702321.76it/s]


Extracting MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272143.06it/s]


Extracting MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4994438.13it/s]


Extracting MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22609714.13it/s]

Extracting MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST/raw



In [7]:
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv1_gn = nn.GroupNorm(2,10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.conv2_gn = nn.GroupNorm(2,20)
        self.fc1 = nn.Linear(320, 50)
        self.fc1_gn =  nn.GroupNorm(2,50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x): 
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1_gn(self.conv1(x)), 2))
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2_gn(self.conv2(x))), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1_gn(self.fc1(x)))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)

In [8]:
class ConvL(nn.Module):
  def __init__(self,in_channels,out_channels,isActive=True, **kwargs):
    super().__init__()
    self.relu=nn.ReLU()
    self.conv = nn.Conv2d(in_channels,out_channels, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels)
    self.isActive = isActive
  def forward(self,x):
    if self.isActive:
        return self.relu(self.bn(self.conv(x)))
    return self.bn(self.conv(x))
class BottleNeckBlock(nn.Module):
  def __init__(self,in_channels,mid_channels,out_channels,stride=1):
    super().__init__()
    self.relu = nn.ReLU()
    if in_channels==64: 
        # 첫 conv
      self.convS = nn.Sequential(
            ConvL(in_channels, mid_channels, kernel_size=1, padding=0),
            ConvL(mid_channels, mid_channels, kernel_size=3, padding=1),
            ConvL(mid_channels, out_channels,isActive=0,kernel_size=1, padding=0)
      )
      self.resi = nn.Conv2d(in_channels,out_channels,kernel_size=1)
    elif in_channels==out_channels:
      self.convS = nn.Sequential(
            ConvL(in_channels, mid_channels, kernel_size=1, padding=0),
            ConvL(mid_channels, mid_channels, kernel_size=3, padding=1),
            ConvL(mid_channels, out_channels,isActive=0, kernel_size=1, padding=0)
      )
      self.resi = nn.Identity()
    else:
      self.convS = nn.Sequential(
            ConvL(in_channels, mid_channels, kernel_size=1, padding=0,stride=2),
            ConvL(mid_channels, mid_channels, kernel_size=3, padding=1),
            ConvL(mid_channels, out_channels,isActive=0, kernel_size=1, padding=0)
      )
      self.resi = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=2)
  def forward(self,x):
    x = self.convS(x) + self.resi(x)
    x = self.relu(x)
    return x
class ResNet50(nn.Module):
    def __init__(self,in_channels=1,num_classes=10):
      super().__init__()
      self.start = nn.Sequential(
          ConvL(in_channels,out_channels=64,isActive=False,kernel_size=7,padding=3,stride=2),
          nn.MaxPool2d(3,2,1)
      )
      self.layer1 = nn.Sequential(
          BottleNeckBlock(64,64,256),
          BottleNeckBlock(256,64,256),
          BottleNeckBlock(256,64,256),
      )
      self.layer2 = nn.Sequential(
          BottleNeckBlock(256,128,512),
          BottleNeckBlock(512,128,512),
          BottleNeckBlock(512,128,512),
          BottleNeckBlock(512,128,512),
      )
      self.layer3 = nn.Sequential(
          BottleNeckBlock(512,256,1024),
          BottleNeckBlock(1024,256,1024),
          BottleNeckBlock(1024,256,1024),
          BottleNeckBlock(1024,256,1024),
          BottleNeckBlock(1024,256,1024),
          BottleNeckBlock(1024,256,1024),
      )
      self.layer4 = nn.Sequential(
          BottleNeckBlock(1024,512,2048),
          BottleNeckBlock(2048,512,2048),
          BottleNeckBlock(2048,512,2048),
      )
      self.avgpool=nn.AvgPool2d(7,2)
      self.fc=nn.Linear(2048,num_classes) 
    def forward(self,x):
      x=self.start(x)
      x=self.layer1(x)
      x=self.layer2(x)
      x=self.layer3(x)
      x=self.layer4(x)
    #   x=self.avgpool(x)
      x=x.reshape(x.shape[0],-1)
      x=self.fc(x)
      return x

In [ ]:
import torch
import torchvision
import torchsummary
net = ResNet50()

torchsummary.summary(net, (1, 28, 28),device='cpu')
# print(net)

In [10]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.cross_entropy(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        print(output)
        test_loss += F.cross_entropy(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
model = ResNet50()
if args['cuda']:
    model.cuda()

optimizer = optim.Adam(model.parameters())

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.470303
Train Epoch: 1 [4992/60000 (8%)]	Loss: 0.659355
Train Epoch: 1 [9984/60000 (17%)]	Loss: 0.531614
Train Epoch: 1 [14976/60000 (25%)]	Loss: 0.447345
Train Epoch: 1 [19968/60000 (33%)]	Loss: 0.597117
Train Epoch: 1 [24960/60000 (42%)]	Loss: 0.391761
Train Epoch: 1 [29952/60000 (50%)]	Loss: 0.496301
Train Epoch: 1 [34944/60000 (58%)]	Loss: 0.513365
Train Epoch: 1 [39936/60000 (67%)]	Loss: 0.497913
Train Epoch: 1 [44928/60000 (75%)]	Loss: 0.408327
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.581885
Train Epoch: 1 [54912/60000 (91%)]	Loss: 0.522180
Train Epoch: 1 [44928/60000 (100%)]	Loss: 0.301533


<ipython-input-10-786c9008d387>:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.5038, Accuracy: 8278/10000 (83%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.346631
Train Epoch: 2 [4992/60000 (8%)]	Loss: 0.381478
Train Epoch: 2 [9984/60000 (17%)]	Loss: 0.297231
Train Epoch: 2 [14976/60000 (25%)]	Loss: 0.426120
Train Epoch: 2 [19968/60000 (33%)]	Loss: 0.705629
Train Epoch: 2 [24960/60000 (42%)]	Loss: 0.283381
Train Epoch: 2 [29952/60000 (50%)]	Loss: 0.423245
Train Epoch: 2 [34944/60000 (58%)]	Loss: 0.349559
Train Epoch: 2 [39936/60000 (67%)]	Loss: 0.406013
Train Epoch: 2 [44928/60000 (75%)]	Loss: 0.263499
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.516194
Train Epoch: 2 [54912/60000 (91%)]	Loss: 0.402127
Train Epoch: 2 [44928/60000 (100%)]	Loss: 0.464516

Test set: Average loss: 0.4160, Accuracy: 8527/10000 (85%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.410008
Train Epoch: 3 [4992/60000 (8%)]	Loss: 0.395986
Train Epoch: 3 [9984/60000 (17%)]	Loss: 0.313661
Train Epoch: 3 [14976/60000 (25%)]	Loss: 0.339705
Train Epoch: 3 [19968/60000 (33%)]	Loss: 0.

In [ ]:
# for epoch in range(1, args['epochs'] + 1):
    test()